In [ ]:
import win32ui
import win32gui
import dde
import time

def fetch_dde_data(service, topic, item):
    try:
        dde_client = dde.CreateServer()
        dde_client.Create("MyClient")
        
        conversation = dde.CreateConversation(dde_client)
        conversation.ConnectTo(service, topic)
        
        # 請求數據
        result = conversation.Request(item)
        
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        # 清理 DDE 連接
        dde_client.Destroy()

# 使用範例
if __name__ == "__main__":
    service = "XQLITE"
    topic = "Quote"
    item = "3715.TW-ID,Name,CompanyPos,Price,DIndexPtsContributionRate,VolumeRatio,TurnoverRatio,PriceChangeRatio,DLOrderValueDiffRatio,MajorOrderBidAskRatio,Low,MonthReturn,QuarterReturn,HalfYearReturn,YearReturn,DXLOrderValueDiffRatio,DetailIndustry,CompanyNews"

    data = fetch_dde_data(service, topic, item)
    if data:
        print("Received data:", data)
        # 在這裡處理接收到的數據
    else:
        print("Failed to fetch data")

In [6]:
import pandas as pd
import _beowFmt as fm
import _beowDDE as bwdde

# 設置顯示選項
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 880)  # 設置顯示寬度
# pd.set_option('display.max_columns', None)  # 顯示所有列
pd.set_option('display.max_rows', 40)  # 顯示所有行
pd.set_option('display.expand_frame_repr', False)  # 不自動換行


def process_dde_data(columns, data, sep):
    # 打印原始數據以檢查格式
    # print("Raw DDE data:")
    # for item in data:
    #     print(item) 

    processed_data = []
    for item in data:
        # 如果數據是單個字符串，嘗試分割
        if isinstance(item, str):
            # 這裡使用製表符（\t）分割，您可能需要根據實際格式調整
            processed_item = item.split(sep)
            if len(processed_item) != len(columns):
                print(len(processed_item),len(columns))
                print(f"Warning: Unexpected number of fields in item: {item}")
                continue
        else:
            print(f"Warning: Unexpected data type for item: {type(item)}")
            continue
        processed_data.append(processed_item)
    
    return pd.DataFrame(processed_data, columns=columns)

def calcX(r):
    volRate, turnOver, whaleSpreadRatio, yoy, mom, net = r["量比"], r["換手率%"], r["大戶差比"], r['yoy'],r['mom'],r['net']
    volRate2, yoymom, net2 = 1, 1, 1

    if (yoy > 2.1 and mom > 2.1):
        yoymom = 2
    if (net > 20):
        net2 = round(net/20,1)
    if volRate >= 1.8:
        volRate2 = volRate

    return round(volRate2 * turnOver * whaleSpreadRatio * yoymom * net2 / 100 , 1)

def fmt_xq2json(r):
    stockId, stockName,k, low, high, close, yesterdayClose, amplitude, estValue, totalValue, volRate, turnOver,\
          roe, ioRate, market, vMa5, ln, whaleSpread, whaleSpreadRatio, \
          cap, pe, pb, yoy, mom, net, dvd, info, shh, inv, x, mn = r["代碼"], r["商品"], r["多空"], r["最低"], r["最高"], r["成交"], r["昨收"], r["漲幅%"], r["估計量"],r["總量"], r["量比"], r["換手率%"], \
            r["ROE%"], r["內外盤%"], r["market"], r["五日量比"], r["融資使用率%"], r["大戶差2"], r["大戶差比"], \
            r["cap"],r["pe"],r['pb'],r['yoy'],r['mom'],r['net'],r["dvd"],r["info"],r['shh'],r['inv'],r['x'],r['mm']
   
    # ln = "" if (ln == '--') else ln   # Finance Used Ratio 融資使用率
    roe = 0 if (roe == '--') else roe
    ioRate = 0 if (ioRate == '--') else ioRate
    estValue = 0 if (estValue == '--') else estValue
    yoy = 0 if (yoy == '--') else yoy
    mom = 0 if (mom == '--') else mom
    net = 0 if (net == '--') else net
    info = '' if (info == '-') else info
    try:
        if low == '0' or high == '0' or yesterdayClose == '0':
            jmp = 0
        else:
            jmp = round(low - yesterdayClose,1) if amplitude > 0 else round(yesterdayClose - high,1)
    except Exception as e:
        print(low, high, yesterdayClose)
        print(f"An error occurred: {e}")
        jmp = 0
    
    sql = f'"id":{stockId},"n":"{stockName}","k":{k},"j":{jmp},"c":{close},"yc":{yesterdayClose},"v":{totalValue},"vE":{estValue},"amp":{amplitude},"vR":{volRate},"turOv":{turnOver},"roe":{roe},"ioR":{ioRate},"ind":"{market}","v5":{vMa5},"ln":"{ln}","wts":{whaleSpread},"wtr":{whaleSpreadRatio},"pe":{pe},"pb":{pb},"yy":{yoy},"mm":{mom},"nt":{net},"dd":"{dvd}","fo":"{info}","sh":"{shh}","iv":"{inv}","x":{x},"mn":"{mn}"'
    return "{" + sql + "},"


def fmt_group_json(r):
    market, vrMean, turnOverMean, k, count, bigSum, bigCount, details = r['market'], r['平均量比'], r['換手率%'], r['多空'], r['筆數'], r['大戶買總額'],r['大戶買多的股票數量'], r['相關股票明細']
        
    sql = f'"market":"{market}","vmn":{vrMean},"tov":{turnOverMean},"k":{k},"cc":{count},"sum":{bigSum},"bigCC":{bigCount},"dls":"{details}"'
    # sql = f'"market":"{market}","dls":"{details}"'
    return "{" + sql + "},"

def Main(topic, ddeDict, stock_Ids, sep):
    service = "XQLITE"

    dde_List = list(ddeDict.keys())   #將dictionary物件轉換成list欄位
    print(dde_List)
    ddeItems = [f"{code}.TW-{",".join(dde_List)}" for code in stock_Ids]
    data = bwdde.fetch_multiple_dde_data(service, topic, ddeItems)

    
    if data:
        df = process_dde_data(dde_List, data, sep)
        # 重命名列名
        df = df.rename(columns=ddeDict)
        

       # 將百分比列轉換為浮點數
        df['ID'] = df['代碼'].astype(int)
        df['內外盤%'] = df['內外盤%'].str.replace('%', '')
        df['漲幅%'] = df['漲幅%'].str.replace('%', '').replace('+', '')
        df['融資使用率%'] = df['融資使用率%'].str.replace('--', '')

        str2Float_columns = ['漲幅%', '成交', '昨收', '最低', '最高', '換手率%', '大單差%', '大戶差比', '量比']
        df[str2Float_columns] = df[str2Float_columns].replace('--', '0').astype(float)        
        df['總量'] = pd.to_numeric(df['總量'])
        df['總量'] = df['總量'].astype(int)
        df['多空'] = df['漲幅%'].apply(lambda x: 1 if x > 0 else 0)
        df['大戶差2'] =df['大戶差'].apply(bwdde.to_billion)           #全部改以億為單位
        df['五日量比'] = df.apply(lambda r: round(float(r['總量']) / float(r['五日均量']), 1) if r['估計量'] == '--' and float(r['五日均量']) != 0 else
                           round(float(r['估計量']) / float(r['五日均量']), 1) if float(r['五日均量']) != 0 else
                           0, axis=1)
        
        #--合併產業資訊--------------------------------------------------------------------------------
        stock_group_file = r"webJson\stock_group.csv"
        stockMarket = pd.read_csv(stock_group_file, encoding='utf-8')
        stockMarket.columns = ["name","id","market"]
        # print(stockMarket.head())
        df = pd.merge(df, stockMarket, left_on="ID", right_on="id")  ## 結合股票名稱
        # print(df)
       
        # df2 = df[(df['大戶差2'] > 0.7) & (df['大戶差比'] > 10)]
        # df2 = df[(df['大戶差比'] > 15) | (df['大戶差比'] < -15)]
        df2 = df[((df['大戶差比'] > 6) & (df['大戶差2'] >= 0.8)) | ((df['大戶差比'] < -1) & (df['大戶差2'] <= -0.6))]
        
        grouped = df2.groupby(['market','多空']).agg({
            '量比': ['mean','count'],
            '換手率%': ['mean'],
            '大戶差2': ['sum','mean','count']
        })
        grouped.columns = ['平均量比', '筆數', '換手率%', '大戶買總額','大戶買平均','大戶買多的股票數量']
        grouped = grouped.round(1)
        

        df_group = grouped[(grouped['大戶買總額'] >= 0.8) | (grouped['大戶買總額'] <= 0)].sort_values(by='大戶買平均', ascending=False)
        # df_group = grouped[grouped['平均大戶差比'] >= 10].sort_values(by='平均量比', ascending=False)
        # df_group = grouped[grouped['平均大戶差比'] >= 10].sort_values(by='平均量比', ascending=False)
        # 統計題材之後，產出對應的股票名稱，以量比由大到小排列，可快速瀏覽...
        df_group['相關股票明細'] = df2.groupby(['market','多空']).apply(lambda x: ','.join(x.sort_values(by='量比', ascending=False)['商品']))
        df_group = df_group.reset_index() # 重置索引，使market成為一個欄位
        df_group['json'] = df_group.apply(fmt_group_json, axis=1)

        #代表性不夠的就予以排除
        # df_filtered = df_group[~((df_group['平均量比'] <= 1.2) & (df_group['筆數'] == 1))]
        # df_filtered = df_filtered[~((df_filtered['多空'] == 1) & (df_filtered['大戶買總額'] < 0))]
        print(df_group)


        targe_file = r"webJson\currStockMarket.json"
        ss = ''.join(df_group['json'].fillna('').astype(str))[:-1]
        fm.write_LogFile(targe_file, f"[{ss}]")
        fm.FtpFile(targe_file, 'static/currStockMarket.json')

        #--合併財務資訊--------------------------------------------------------------------------------
        financeData_file = r'webJson\\financeData.csv'
        financeData = pd.read_csv(financeData_file, encoding='utf-8')
        financeData = financeData.fillna('')
        financeData.columns = ["id","cap","pe","pb","yoy","mom","net","dvd","info","shh","mm","inv"]
        # print(financeData.head(60))
        df = pd.merge(df, financeData, left_on="ID", right_on="id")  ## 結合股票名稱
        df['x'] = df.apply(calcX, axis=1)
        df['mm'] = df['mm'].str.replace('2024/', '') #營收月份 2024/08
        df['json']= df.apply(fmt_xq2json, axis = 1) 
        # print(df[df["ID"]==4768])

        targe_file = r"\webJson\currentMaxValue.100.json" #產出json資料到WebJson目錄下
        ss = ''.join(df['json'].fillna('').astype(str))[:-1]
        fm.write_LogFile(targe_file, f"[{ss}]")     #送出到網站    
        fm.FtpFile(targe_file, 'static/currentMaxValue.100.json')
    else:
        print("Failed to fetch data")


# 追蹤股票號碼
stock_File = r"webJson\a013_stockIds.txt"
stock_Ids = open(stock_File, "r").read().split(",")
dde_basic_dict={
            'ID': '代碼', 
            'Name': '商品',
            'Price': '成交',
            'TotalVolume': '總量',
            'VolumeRatio': '量比',
            'TurnoverRatio': '換手率%',
            'PreClose': '昨收',
            'PriceChangeRatio': '漲幅%',
            'DLOrderValueDiffRatio': '大單差%',
            'MajorOrderDif': '大戶差',
            'MajorOrderDifRatio': '大戶差比',
            'Low': '最低',
            'High': '最高',
            'ROE': 'ROE%',
            'InOutRatio': '內外盤%',
            'EstimatedTotalVolume': '估計量',
            '5DayAvgVol': '五日均量',
            'FinanceUsedRatio': '融資使用率%',
        }
Main("Quote", dde_basic_dict, stock_Ids, ";")

                # 'DXLOrderValueDiffRatio',      #特大單差
                # 'DIndexPtsContributionRate',   #佔大盤比
                # 'MonthReturn',    #1月%
                # 'QuarterReturn',  #一季%
                # 'HalfYearReturn', #半年%
                # 'YearReturn',     #一年%
                # 'RevenueMonth','EstimatedTotalVolume','5DayAvgVol']
                
# HistColumns = ["F001","F002","F009"]
# HistItems = [f"{code}.TW-day-1-{",".join(HistColumns)}" for code in stock_Ids]
# Main("Hist", HistColumns, HistItems, ",")


['ID', 'Name', 'Price', 'TotalVolume', 'VolumeRatio', 'TurnoverRatio', 'PreClose', 'PriceChangeRatio', 'DLOrderValueDiffRatio', 'MajorOrderDif', 'MajorOrderDifRatio', 'Low', 'High', 'ROE', 'InOutRatio', 'EstimatedTotalVolume', '5DayAvgVol', 'FinanceUsedRatio']


C:\Users\danie\AppData\Local\Temp\ipykernel_23252\1256609333.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_group['相關股票明細'] = df2.groupby(['market','多空']).apply(lambda x: ','.join(x.sort_values(by='量比', ascending=False)['商品']))


         market  多空  平均量比  筆數  換手率%  大戶買總額  大戶買平均  大戶買多的股票數量          相關股票明細                                                    json
0      線上遊戲     1       2.5     1      2.1        16.1        16.1                   1                  鈊象  {"market":"線上遊戲","vmn":2.5,"tov":2.1,"k":1,"cc... 
1        IC設計     1       2.2     2      1.5        24.2        12.1                   2         金麗科,聯發科  {"market":"IC設計","vmn":2.2,"tov":1.5,"k":1,"cc...   
2      液冷三雄     1       3.5     3      4.2        22.8         7.6                   3        建準,奇鋐,雙鴻  {"market":"液冷三雄","vmn":3.5,"tov":4.2,"k":1,"cc... 
3         CoWoS     1       2.2     1      7.0         7.5         7.5                   1                  志聖  {"market":"CoWoS","vmn":2.2,"tov":7.0,"k":1,"c...     
4      傳動元件     1       4.5     1      2.9         5.3         5.3                   1                  上銀  {"market":"傳動元件","vmn":4.5,"tov":2.9,"k":1,"cc... 
5    光通訊設備     1       1.4     2     24.3        10.2         5.1   

In [1]:
import pandas as pd
from datetime import datetime
import _beowFmt as fm
import _beowDDE as bwdde
import numpy as np

# 設置顯示選項
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 880)  # 設置顯示寬度
# pd.set_option('display.max_columns', None)  # 顯示所有列
pd.set_option('display.max_rows', None)  # 顯示所有行
pd.set_option('display.expand_frame_repr', False)  # 不自動換行


def process_dde_data(columns, data, sep):
    # 打印原始數據以檢查格式
    # print("Raw DDE data:")
    # for item in data:
    #     print(item) 

    processed_data = []
    for item in data:
        # 如果數據是單個字符串，嘗試分割
        if isinstance(item, str):
            # 這裡使用製表符（\t）分割，您可能需要根據實際格式調整
            processed_item = item.split(sep)
            if len(processed_item) != len(columns):
                print(len(processed_item),len(columns))
                print(f"Warning: Unexpected number of fields in item: {item}")
                continue
        else:
            print(f"Warning: Unexpected data type for item: {type(item)}")
            continue
        processed_data.append(processed_item)
    
    return pd.DataFrame(processed_data, columns=columns)


# 假設 today 是今天的日期
# today = pd.to_datetime('2023-10-01')

# 定義一個函數來處理日期替換和轉換
def process_date(date_str):
    if date_str == '--':
        date_str = '2022/1/1'
    return pd.to_datetime(date_str, format='%Y/%m/%d', errors='coerce')

# 定義一個函數來檢查日期並轉換為字符串格式
def check_date(date, today):
    if pd.isna(date):
        return ''
    if date.date() > today:
        return date.strftime('%Y/%m/%d')
    else:
        return ''


def Main(topic, ddeDict, stock_Ids, sep):
    service = "XQLITE"

    dde_List = list(ddeDict.keys())   #將dictionary物件轉換成list欄位
    # print(dde_List)
    ddeItems = [f"{code}.TW-{",".join(dde_List)}" for code in stock_Ids]
    data = bwdde.fetch_multiple_dde_data(service, topic, ddeItems)
    # print(data)
    today = datetime.now().date()
    
    if data:
        df = process_dde_data(dde_List, data, sep)
        df = df.rename(columns=ddeDict)  # 重命名列名

        # 篩選出日期大於今天的行，否則設為空字串
        # today = pd.to_datetime(today)
        # df['除息日'] = df['除息日'].str.replace('--','2022/1/1')
        # df['除息日'] = pd.to_datetime(df['除息日'], format='%Y/%m/%d')
        # df['除息日'] = df['除息日'].apply(lambda x: x if x.date() > today else '')
        # df['除息日'] = df['除息日'].apply(lambda x: x.strftime('%Y/%m/%d') if isinstance(x, pd.Timestamp) else '')

        # df['法說會日'] = df['法說會日'].str.replace('--','2022/1/1')
        # df['法說會日'] = pd.to_datetime(df['法說會日'], format='%Y/%m/%d')
        # df['法說會日'] = df['法說會日'].apply(lambda x: x if x.date() > today else '')
        # df['法說會日'] = df['法說會日'].apply(lambda x: x.strftime('%Y/%m/%d') if isinstance(x, pd.Timestamp) else '')

        # 將 '--' 替換為 '2022/1/1'，然後轉換為 datetime 格式
        df['除息日'] = df['除息日'].apply(process_date)
        df['除息日'] = df['除息日'].apply(lambda x: check_date(x, today))
        df['法說會日'] = df['法說會日'].apply(process_date)
        df['法說會日'] = df['法說會日'].apply(lambda x: check_date(x, today)) # 使用 apply 來檢查日期，並將結果轉換為字符串格式
        df['股東會日'] = df['股東會日'].apply(process_date)
        df['股東會日'] = df['股東會日'].apply(lambda x: check_date(x, today))

        df['公司動態'] = df['公司動態'].str.replace('--', '')
        str2Float_columns = ['PE', 'PB', '當月營收YOY', '當月營收MOM', '稅後淨利成長率%']
        df[str2Float_columns] = df[str2Float_columns].replace('--', '0').astype(float)    
        print(df.head())

        csv_file = r'D:\project\stockDataLab\Lab\data\\webJson\\financeData.csv'
        df.to_csv(csv_file, index=False)   #這邊產生CSV檔提供給下面的去合併
    else:
        print("Failed to fetch data")


# 追蹤股票號碼
stock_Ids = open("data\\webJson\\a013_stockIds.txt", "r").read().split(",")
dde_fin_dict = {
            'ID': '代碼',
            # 'Name': '商品',
            'Capital': '股本',
            'PERatio': 'PE', 
            'PBRatio': 'PB',  
            'MonthlyNetSalesYoY': '當月營收YOY',
            'TFXRatio':            '當月營收MOM',
            'NetIncomeGrowthRate': '稅後淨利成長率%',
            'ChipsField25':    '除息日', #dvd ex-dividend date
            # 'DetailIndustry':  '細產業', #這太長了 被我的題材標籤取代
            'CompanyNews':  '公司動態',
            'DateofMeetingofShareHolders': '股東會日',
            'RevenueMonth': '營收月份',
            'InvestorConferenceDate':'法說會日',
        }

Main("Quote", dde_fin_dict, stock_Ids, ";")


   代碼    股本     PE    PB  當月營收YOY  當月營收MOM  稅後淨利成長率% 除息日           公司動態 股東會日 營收月份 法說會日
0  1101  755.12  33.03  1.06        52.55         2.47            10.29         9月營收133.25億元           2024/09         
1  1102  354.66  13.70  0.92        -2.92        -3.53            -6.13          9月營收64.97億元           2024/09         
2  1110   57.20  91.67  1.38        70.78       -11.60            34.64           9月營收2.89億元           2024/09         
3  1210   89.48  11.33  1.88       -11.76         6.13            -2.66                                     2024/08         
4  1216  568.20  26.91  4.03         7.10         3.66            -3.75                                     2024/08         


In [ ]:
from dotenv import load_dotenv
import os

# 加載 .env 文件
load_dotenv()

hostname = os.getenv('FTP_HOSTNAME')
username = os.getenv('FTP_USERNAME')
password = os.getenv('FTP_PASSWORD')

print(f"Hostname: {hostname}")
print(f"Username: {username}")
print(f"Password: {password}")